# Standardizing data for machine learning

### What is standardization?

Standardization is a preprocessing method used to transform continuous data to make it look normally distributed. In scikit-learn, this is often a necessary step, because many models assume that the data you are training on is normally distributed, and if it isn't, you risk biasing your model. You can standardize your data in different ways, but in this workshop we're going to talk about two methods: log normalization and scaling. 

There are a few different scenarios in which you want to standardize your data. First, if you're working with any kind of model that uses a linear distance metric or operates in a linear space like k-nearest neighbors, linear regression, or k-means clustering, the model is assuming that the data and features you're giving it are related in a linear fashion, or can be measured with a linear distance metric. There are a number of models that deal with nonlinear spaces, but for those models that are in a linear space, the data must also be in that space.

The case when a feature or features in your dataset have high variance is similar. This could bias a model that assumes the data is normally distributed. If a feature in your dataset has a variance that's an order of magnitude or more greater than the other features, this could impact the model's ability to learn from other features in the dataset. Also, modeling a dataset that contains continuous features that are on different scales is another scenario to watch out for. For example, consider a dataset that contains a column related to height and another related to weight. In order to compare these features, they must be in the same linear space, and therefore must be standardized in some way.

### Log normalization

Log normalization is a method for standardizing your data that can be useful when you have a particular column with high variance. Log normalization applies a log transformation to your values, which transforms your values onto a scale that approximates normality, an assumption about data that a lot of machine learning models make. The method of log normalization we're going to work with in Python takes the natural log of each number, which is simply the exponent you would raise above the mathematical constant _e_ (approximately equal to 2.718) to get that number.  

Log normalization is a good strategy when you care about relative changes in a linear model, when you still want to capture the magnitude of change, and when you want to keep everything in the positive space. It's a nice way to minimize the variance of a column and make it comparable to other columns for modeling.

Let's see how this works in Python. Here we have a small dataset where one of the columns, `col2`, has high variance. We can check the variance across a set of data using the `.var()` method:

In [ ]:
import pandas as pd

var_df = pd.DataFrame({"col1": [1.0, 1.2, 0.75, 1.6], 
                       "col2": [3.0, 45.5, 28.0, 100.0]})

var_df.____

Applying log normalization to data in Python is fairly straightforward. We can use the `np.log()` function from NumPy to log normalize `col2`.

In [ ]:
import numpy as np

var_df["col2_log"] = ____(var_df["col2"])
var_df

If we check the variance of both `col1` and the `col2_log`, we can see that the variances are much closer together now:

In [ ]:
____(var_df[["col1", "col2_log"]])

### Scaling for feature comparison

Scaling is a method of standardization that's most useful when you're working with a dataset that contains continuous features that are on different scales, and you're using a model that operates in some sort of linear space (like linear regression or k-nearest neighbors). Feature scaling transforms the features in your dataset so they have a mean of zero and a variance of one. This will make it easier to linearly compare features. This is a requirement for many models in scikit-learn.

Let's take a look at another dataframe.

In [ ]:
scale_dict = {"col1": [1.0, 1.2, 0.75, 1.6],
              "col2": [48.0, 45.5, 46.2, 50.0],
              "col3": [100.0, 101.3, 103.5, 104.1]}

scale_df = pd.DataFrame(scale_dict)

scale_df

In each column, we have numbers that are relatively close within the column, but not across columns. If we look at the variance, it's relatively low across columns:

In [ ]:
scale_df.____

To better model this data, scaling would be a good choice here.

Scikit-learn has a variety of scaling methods, but we're only going to focus on the `StandardScaler()` method. This method works by removing the mean and scaling each feature to have unit variance. There's a simpler scale function in scikit-learn, but the benefit of using `StandardScaler()` is that you can apply the same transformation on other data, like a test set, or new data that's part of the same set, for example, without having to rescale everything.

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = ____

Once we have the standard scaler method, we can apply the `fit_transform()` function on the DataFrame, setting the `columns` parameter to `scale_df.columns`. Here we're keeping it as a DataFrame to see the results more easily.

In [ ]:
df_scaled = pd.DataFrame(scaler.____(scale_df), columns=____)

Taking a look at the DataFrame, it has now been scaled, and the variance is now equivalent across columns:

In [ ]:
print(df_scaled)
print("\n")
print(df_scaled.____)

### Your turn!

Let's try out two situations where you'd want to use these standardization techniques on a dataset, beginning with log normalization.

First, read in the csv file. Then take a look at the feature variance in the `wine_types` dataset:

In [ ]:
dir_string = "../../datasets/"
wine_types = ____(dir_string + "wine_types.csv")

wine_types.____

We can see that the `Proline` column has much higher variance than anything else in the dataset. This column would be a good candidate for log normalization. Let's apply the `np.log()` function to the `Proline` column:

In [ ]:
wine_types["Proline_log"] = ____

wine_types["Proline_log"].____

We can see the variance has been reduced, which will make feature comparisons easier.

Next, apply scaling to the `wine_types` dataset as well. Let's say that we want to use the `Ash`, `Alcalinity of ash`, and `Magnesium` columns to train a linear model, but it's possible that these columns are all measured in different ways. 

Taking a quick look at these columns with the `.describe()` method:

In [ ]:
wine_types[["Ash", "Alcalinity of ash", "Magnesium"]].____

We can see that these three columns are on different scales—the means, mins, and maxes are wildly different. Since we can see that these columns are all on different scales, let's standardize them in a way that allows for use in a linear model.

First we'll create a new `StandardScaler()` method:

In [ ]:
wine_scaler = ____

Then we'll apply it to the subset of the `wine_types` dataset using `fit_transform()`:

In [ ]:
wine_subset = wine_types[["Ash", "Alcalinity of ash", "Magnesium"]]

wine_subset_scaled = wine_scaler.____

And finally, let's take a look at the scaled data and its variance by transforming it back into a DataFrame:

In [ ]:
wine_subset_scaled_df = pd.DataFrame(wine_subset_scaled, columns=wine_subset.columns) 

print(wine_subset_scaled_df.head())
print("\n")
print(wine_subset_scaled_df.____)

The data has been scaled and the variance is equal across columns.

### knn refresher

Let's do a quick refresher on k-nearest neighbors and scikit-learn in general, which we'll be using throughout the workshop. 

K-nearest neighbors is a model that classifies data based on its distance to training set data. A new data point is assigned a label based on the class that the majority of surrounding data points belongs to, using a distance metric like the Euclidean distance metric. 

Let's walk through an example of k-nearest neighbors and the scikit-learn workflow. First, let's generate a toy dataset using scikit-learn's `make_classification()` method, which generates a normally-distributed dataset you can use for classification problems. There are a number of ways you can customize the generated dataset, but we're going to keep it simple here. 

The parameters filled in below are:
- `n_samples`: the number of samples (rows) in the dataset. The default is 100; here it's increased to 1000.
- `n_features`: the number of features (columns) in the dataset. Instead of the default 20, let's keep it very simple at 3 features.
- `n_classes`: the number of class labels. We'll keep it at the default 2. 
- `n_redundant`: the number of redundant features—features that aren't informative for classification—which is useful if you want to test out dimensionality reduction methods, for example. We're not doing that here, so we'll set it to 0.

The function generates both the X and y sets, so we'll store those sets in the `X_gen` and `y_gen` variables.

In [ ]:
from sklearn.datasets import make_classification

X_gen, y_gen = make_classification(n_samples=1000, n_features=3, n_classes=2, n_redundant=0)

Now that we have the data, let's build our model. We'll need both `train_test_split`, to split up our dataset before training, and `KNeighborsClassifier()`, to train the k-nearest neighbors model.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

Let's split up our data into training and test sets:

In [ ]:
X_train, X_test, y_train, y_test = ____

Once we've done that, we can create our `KNeighborsClassifier()` model, and then use `.fit()` to fit the model to our training data.

In [ ]:
knn = ____
knn.____(X_train, y_train)

The last step we want to take is to check the accuracy of our classifier. There are a multitude of ways you can validate the accuracy of your mode, but we'll just use `KNeighborsClassifier()`'s built in `.score()` method, which takes your `X` test data, predicts its `y` labels, and then compares the predicted `y` to the true values of `y` you pass in as the second parameter. The output is the percentage in the test `X` that received an accurate prediction.

In [ ]:
knn.____(X_test, y_test)

### Your turn!

In this section, we're going to build a k-nearest neighbors model on both non-scaled and scaled data, to give you a better idea of how that impacts model performance.

#### Wine_types modeling: non-scaled data

Let's first take a look at the accuracy of a k-nearest neighbors model on the `wine_types` dataset without standardizing the data. We're going to use the features in this dataset to predict the `Type` of wine, of which there are three labels: 1, 2, and 3.

Let's break out `Type` into its own `y_wine` dataset as our predicted class, and we'll keep the rest of the features in `X_wine`.

In [ ]:
X_wine = wine_types.drop("Type", axis=1)
y_wine = wine_types["Type"]

We've already previously imported `train_test_split` so let's split up our data:

In [ ]:
X_train, X_test, y_train, y_test = ____

Now we can create a new `KNeighborsClassifier()` model:

In [ ]:
knn_wine_unscaled = ____

And then we can fit the model to our training set and then take a look at its accuracy:

In [ ]:
knn_wine_unscaled.____

knn_wine_unscaled.____

The accuracy isn't terrible, but it's possible that we can improve our model by scaling our data!

#### Wine_types modeling: scaled data

The accuracy score on the unscaled `wine_types` dataset was decent, but we can likely do better if we scale the dataset. The process is mostly the same as the previous exercise, with the added step of scaling the data.

Let's apply the `StandardScaler` method to our `wine_types` feature set:

In [ ]:
wine_scaler = ____

X_scaled = wine_scaler.____

Once again, we'll split up the data:

In [ ]:
X_train_scaled, X_test_scaled, y_train_scaled, y_test_scaled = ____

Then fit the `KNeighborsClassifier` model to the data and take a look at the accuracy score:

In [ ]:
knn_wine_scaled = ____
knn_wine_scaled.____

knn_wine_scaled.____

We can see that there's been a dramatic increase in the accuracy of our model, simply by scaling our dataset.